# NB_SVM_LR_SA_IMDB_TFIDF_RS

## Model 1:
1. UseFusing Naive Bayes and SVM/LR
2. Do Sentiment Analysis (SA)
3. TF-IDF
4. Refined Stopwords (RS)

The notebook will have the following structure:
1. Importing libraries and data
2. Exploratory data analysis
3. Tokenization and training/validation split
4. Naive Bayes with TF-IDF
5. Logistic Regression/SVC with NB features
7. Inference and submission



In [1]:
## Importing libraries and data
#from fastai.nlp import *
import fastai
#fastai.download('nlp')


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd, numpy as np
#from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pathlib import Path
import os, re, string

In [3]:
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import f1_score

In [4]:
path = Path('C:/Users/xbabu/Babu/pers/PhD/MIST.7060 Data Analytics/Project/data/IMDB')
names = ['neg', 'pos']
# path = Path('../data/Toxic')
names = ['neg', 'pos']
os.listdir(path)

['test', 'train']

In [5]:
#os.listdir(path/'train/pos')[0:20]
#

In [6]:

def texts_from_files(src, names):
    texts,labels = [],[]
    for idx,name in enumerate(names):
        path = os.path.join(src, name)
        t = [o.strip() for o in open(path, encoding = "ISO-8859-1")]
        texts += t
        labels += ([idx] * len(t))
    return texts,np.array(labels)

def texts_from_folders(src, names):
    texts,labels = [],[]
    MAX_DOCS = 10000
    for idx,name in enumerate(names):
        path = os.path.join(src, name)
        i = 1
        for fname in sorted(os.listdir(path)):
            fpath = os.path.join(path, fname)
            texts.append(open(fpath, encoding="utf8").read())
            labels.append(idx)
            i = i + 1
            if i > MAX_DOCS: 
                break
    return texts,np.array(labels)

#def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [7]:
from sklearn.datasets import load_files
train, train_y = texts_from_folders(path/'train',names)
val, val_y = texts_from_folders(path/'test',names)

#??load_files
#files = load_files('C:/Users/xbabu/Babu/pers/PhD/MIST.7060 Data Analytics/Project/data/IMDB/train', encoding='utf-8')

In [8]:
import nltk
#nltk.download('punkt')

import string
from nltk.stem import WordNetLemmatizer

def lemma_tokenizer(corpus): # a method to lemmatize corpus
    corpus = ''.join([ch for ch in corpus if ch not in string.punctuation]) # remove punctuation
    tokens = nltk.word_tokenize(corpus)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

stopwords = nltk.corpus.stopwords.words('english')
stopwords.remove('but')
stopwords.remove('no')
stopwords.remove('not')
#print(stopwords)

In [9]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
# Using trigram features actually is going to turn out to make
# NB and logistic regression quite a lot better
# max_features=1000000, the CountVectorizer will sort the vocabulary by 
# how often everything appears whether it be unigram/bigram/trigram 
# and it will cut it off after the defined count (1000000)
# most common ngrams
#veczr = CountVectorizer(ngram_range=(1,2), tokenizer=tokenize, max_features=800000)
#veczr = CountVectorizer(lowercase=False, stop_words='english', tokenizer=re_tokenizer, ngram_range=(1,2), min_df=2, max_df=0.95, max_features=1000000)
#veczr = CountVectorizer(lowercase=False, tokenizer=lemma_tokenizer, stop_words=stopwords, ngram_range=(1,2), min_df=2, max_df=0.8, max_features=2000000)
vec_tfidf = TfidfVectorizer(ngram_range=(1,4), tokenizer=lemma_tokenizer,  stop_words=stopwords, lowercase=True,
               min_df=4, max_df=0.8, strip_accents='unicode', sublinear_tf=True)
#trn_term_doc = veczr.fit_transform(train)
#val_term_doc = veczr.transform(val)


In [32]:
trn_term_doc = vec_tfidf.fit_transform(train)
val_term_doc = vec_tfidf.transform(val)


In [33]:
#print(trn_term_doc)
#trn_term_doc_dense = trn_term_doc.toarray()
trn_term_doc
#trn_term_doc_dense

<20000x121974 sparse matrix of type '<class 'numpy.float64'>'
	with 3018009 stored elements in Compressed Sparse Row format>

In [34]:
dictionary = vec_tfidf.get_feature_names()
#dictionary
#pd.DataFrame(trn_term_doc_dense, columns=dictionary)

In [35]:
#w0 = set([o.lower() for o in train[1].split(' ')])
#w0

In [36]:
#train_y

In [37]:
#print(len(w0))
print(f'Training set size: {len(train)}')
print(f'Test set size: {len(val)}')
#print(ctvec.vocabulary_['writers'])
#trn_term_doc[1, 934]

Training set size: 20000
Test set size: 20000


# 2. Naive Bayes

We define the **log-count ratio** $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive has a feature divided by the number of positive documents.


In [20]:
from time import time
t0 = time()
x = trn_term_doc
y = train_y

# grab the rows where the dependent varaible is 1, 
# then we sum over the rows to get the total word count 
# for that feature across all the documents 
# plus 1 to consider the unreviewed probability
p = x[y==1].sum(0)+1
# grab the rows where the dependent varaible is 0
q = x[y==0].sum(0)+1
# it is nicer to take the log to add things together 
# rather than mulitply them together
# and once you like multiply enough of these things together it's 
# going to get kind of so close to zero that you will probably run out of
# floating-point, so we take the log of the ratios and 
# we then multiply that or  
# add that to the ratio of the whole class probabilities
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))
#p
#p.sum()
#r
# So in order to say for each document multiply the
# bayes probabilities by the counts, we can just use matrix multiply
# and then to add on the log of the class ratios we can just use + b
# so we end up with something that looks a lot like our logistic regression.
# but we are not learning anything, we are just calculating it using this
# theoretical model
pre_preds = val_term_doc @r.T + b
# Now we compare that as to whether it is bigger or smaller
# smaller than zero, not one anymore because we are now in log space 
preds = pre_preds.T>0
# then we can compare that to the mean and we can say
# how much it is accurate
# so NB is not nothing and it gave us something. 
(preds == val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW NB matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF NB matrix:
Accuracy: 0.87455
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8510                1490
actual negative                1019                8981
precision = 0.8930632805121209 , recall = 0.851 , F1-score = 0.8715243996108352
Elapsed Time: 0.075 seconds.


Formula for Naive Bayes

Binarized Naive Bayes

In [22]:
# Turns out that the version where we are actually looking at
# how often a word appears like "absurd" appeared twice. It turns out
# at least for this problem and quite often it doesn't matter
# whether "absurd" appeared twice or once
# all that matters is it appeared so what we tend to d here is
# take the term document matrix and do doc sign.
# doc.sign replaces anything positive as 1 and 
# anything negative with -1 so it binarizes it
# it gives a better result
t0 = time()
pre_preds = val_term_doc.sign() @r.T + b
#preds
preds = pre_preds.T>0
(preds == val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF NB.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF NB.sign() matrix:
Accuracy: 0.8719
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8312                1688
actual negative                 874                9126
precision = 0.9048552144567821 , recall = 0.8312 , F1-score = 0.8664651308245596
Elapsed Time: 0.063 seconds.


## 2.1 Logistic Regression
instead of using the coefficients r.T, let us learn them using Logistic Regression

In [24]:
# In theory NB sounds ok, but rather than assuming that
# we should use these coefficients r.T why don't we learn them
# let su create a logistic regression
m = LogisticRegression(C=0.1, dual=False)
# let's fit some coefficients that is going to 
# literally give us with exactly the same functional form we have above
# but now rather than using the theoretical r and b, we are going to
# calculate based on the logistic regression and that is better
# Theoretical model are never going to be accurate 
# pretty much as a data-driven model unless we are dealing with some
# physics
# It is better to learn the coefficients and calculate
t0 = time()
m.fit(x,y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF LR :\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF LR :
Accuracy: 0.86365
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8679                1321
actual negative                1406                8594
precision = 0.8605850272682202 , recall = 0.8679 , F1-score = 0.8642270351008216
Elapsed Time: 0.629 seconds.


In [25]:
# the term document matrix is much wider than it is tall,
# there is a reformulation mathematically, equivalent reformulations of
# logistic regression that happens to be a lot faster when it is wider
# than it is tall
t0 = time()
m = LogisticRegression(C=0.1, dual=False)
m.fit(trn_term_doc.sign(),y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF LR.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF LR.sign() matrix:
Accuracy: 0.89055
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8918                1082
actual negative                1107                8893
precision = 0.889576059850374 , recall = 0.8918 , F1-score = 0.8906866416978777
Elapsed Time: 1.567 seconds.


Now the prediction is going to be a coefficient for every term where it is about 75000 terms in their vocabulary and that seems like a lot of coefficients given that we only got 25,000 reviews so we should try regularizing this. Smaller value to the parameter is more regularizing.
Turn on the regularized version by setting C = 0.1
avoid overfitting with L2 regularization with binarized

In [26]:
#m = LogisticRegression(C=0.1, dual=False)
#m.fit(trn_term_doc.sign(),y)
#preds = m.predict(val_term_doc.sign())
#(preds==val_y).mean()

# you wouldn't think like 75000 parameters for 25,000 documents it is likely
# to overfit indeed it did over fit so it (C=0.1) is adding
# L2 regularization to avoid overfitting
# L2 is weight square (l2 makes things zero but if two things are highly
# correlated then L2 regularization will make move them both
# down together, it won't make one of them zero and one of them non-zero) and 
# L1 is absolute value of weight; L1 has a regularization actually has the 
# property that it will try to make as many things zero as possible
# where as L2 regularization has a property that it tends to make
# kind of everything smaller.
# But we don't consider that difference in modern ML and we try to 
# directly interpret the coefficients and we try to understand the models
# through interrogation using the kind of techniques 
# The reason why we care about L1 vs L2 is simply like which one
# ends up with a better error on the vali dation set
# In this case, the L2 turns out to be a lot faster because you can't use
# (dual=True) unless you have L2 and L2 is a default
# 


In [27]:
from time import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1)
svc.fit(x,y)
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
#t0 = time()
preds = svc.predict(val_term_doc)
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF SVM matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF matrix:
Accuracy: 0.8962
Results with TFIDF SVM matrix:
Accuracy: 0.8962
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8878                1122
actual negative                 954                9046
precision = 0.9029698942229455 , recall = 0.8878 , F1-score = 0.8953206938281565
Elapsed Time: 994.287 seconds.


In [28]:
from time import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1)
svc.fit(trn_term_doc.sign(),y)
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
#t0 = time()
preds = svc.predict(val_term_doc.sign())
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF SVM.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))


Results with TFIDF matrix:
Accuracy: 0.8785
Results with TFIDF SVM.sign() matrix:
Accuracy: 0.8785
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8747                1253
actual negative                1177                8823
precision = 0.8813986295848448 , recall = 0.8747 , F1-score = 0.878036538847621
Elapsed Time: 1400.764 seconds.


In [38]:
x = trn_term_doc.sign()
y = train_y
# x[y==1].sum(0) is to calculate the ratio of each feature in positive and no negative by dividing the toekn counts.

val_x = val_term_doc.sign()
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [39]:
################ NB-LR #############################
# take the term-document matrix and multiply by r
# every where 0 appears, 0 appears in x_nb and 
# every where 1 appears in TD, the r appears there
t0 = time()
x_nb = x.multiply(r)
# we are going to use x_nb as independent variables instead in the
# logistic regression
m = LogisticRegression(dual=False, C=0.1)
m.fit(x_nb, y)

# do the samething for the validation set and get the predictions
# to get a better result
val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
#(preds.T==val_y).mean()
print('Results with TFIDF NB-LR matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

# regularization we start out with our 
# Loss = Cross entropy loss based on the predictions and the actuals + penalty
# L = X.C.(XW, y) + a W^2
# If you weights a large then that a.W^2 gets bigger and
# it drowns out X.C.(XW, Y) and we want it to be a good fit
# so we want to have as little regularization going on as we can get away with
# so we want to have lesser weights
# the weights that are close to zero were kind of not there.

# x_nb is creating kind of variation in weights that is going to cause that
# the W^2 to go up 
# When the input is already mulitplied by r is saying penalize things
# where we are varying from our NB prior
# I don't like to rely on the theory but if I have some theory then maybe
# we can use that as our starting point rather than starting off by assuming
# everything's equal, so our prior said we got this model quite Naive Bayes,
# and NB model said if NB assumptions were correct then r is the correct
# coefficient in this specific formulation because our prior is based
# on that theory

# that meeans, in a ML technique we can impute them with this kind of 
# Bayesian sense by starting out by incorporating the theoretical 
# expectations into the data that we feed to our model. 
# When we to do so, we don't need to regularize as much.
# If we regularize a lot, 

# C=1e-5 is the reciprocal of the amount of vectorization penalty so 
# we add lots of regularization by making C value very small
# that really hurts the accuracy, because it is really trying hard to get
# those weights down, the loss function is overwhelmed by the need to 
# reduce the weights and the need to make it predictive is 
# kind of now seen as totally unimportant
# So don't push the weights down so that you end up ignoring the terms
# but instead push them down so that you try to get rid of ignore 
# differences from our expectationi based on the NB formulation

# In deep learning we can get 94% accuracy,, but still particularly for a
# linear technique that's easy, fast and intuitive, this is pretty good and 
# we used trigrams instead of bigrams 

# Summary, it is mathematically equivalent to a normal logistic regression
# and the differene is actually in the regularization 


# Regularized Logistic Regresssion
# LR is prone to overfitting if you fit it with a very high order polymomial 
# features, like using a sigmoid function and it may end up with the hypothesis 
# whose decision boundary is a overly complex and extremely contorted function 
# that is not really a great hypothesis for a training set
# Also, in general, LR with a lot of features, it can end up with overfitting
# So we need to change the cost function of LR by adding the term lambda over 2m and sum from j = 1 to n for Theta j square
# This has the effect of penalizing the parameters Theta 1, 2 and so on...
# Even if we use high order poloymial with a lot of parameters, as long as we apply regularization and keep the parameters small
# the decision boundary will be more reasonable for separating out the positive and negative examples.
# Tuning parameter -- Lambda helps us explore how much we put emphasis on fitting the data versus making the magnitude of coefficient small
# When lambda is 0, we might get the standard likelihood solution i.e. unpenalized MLE solution
# When the lambda is infinity, the optimization becomes only care about penalizing the large coefficients of the parameters
# That means we are not fitting the data at all because of setting all the parameters to 0.
# We need a optimzied lambda which would balance the overfit and the magnitude of the parameter of the coefficients
# Finding the optimized lambda is called L2 regularized logistic regression
# Bias-Variance tradeoff is controlled by lambda
# When lambda is large can lead to large bias since theycan't fit the data very well and we have low variance
# When lambda is small, we get very good fit to the training data, so we have low bias, but we can have a very high variance
# if the data changed a little bit we get a completely different decision boundary 

# large can lead to large bias since theycan't fit the data very well and we have low variance



Results with TFIDF NB-LR matrix:
Accuracy: 0.9035
Confusion matrix:
                  predicted positive  predicted negative
actual positive                9049                 951
actual negative                 979                9021
precision = 0.9023733546071001 , recall = 0.9049 , F1-score = 0.9036349111244258
Elapsed Time: 1.233 seconds.


In [40]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1e5)
svc.fit(x_nb, y)
print("Elapsed Time: %0.3f seconds." % (time() - t0))

t0 = time()
preds = svc.predict(val_x_nb)
print('Results with TFIDF NB-SVM matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Elapsed Time: 554.101 seconds.
Results with TFIDF NB-SVM matrix:
Accuracy: 0.8821
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8738                1262
actual negative                1096                8904
precision = 0.8885499288183852 , recall = 0.8738 , F1-score = 0.8811132398910961
Elapsed Time: 137.037 seconds.


## 2.2 Trigram with NB Features
Logistic regression with Naive Bayes features descrived here. For every document we compute binarized features as described above, but this time we use bigrams and trigrams too. Each feature is a log-count ratio. A logistic regression model is then trained to predict sentiment.

In [85]:
import nltk
#nltk.download('punkt')

import string
from nltk.stem import WordNetLemmatizer

def lemma_tokenizer(corpus): # a method to lemmatize corpus
    corpus = ''.join([ch for ch in corpus if ch not in string.punctuation]) # remove punctuation
    tokens = nltk.word_tokenize(corpus)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

stopwords = nltk.corpus.stopwords.words('english')
stopwords.remove('but')
stopwords.remove('no')
stopwords.remove('not')
#print(stopwords)


In [101]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
# Using trigram features actually is going to turn out to make
# NB and logistic regression quite a lot better
# max_features=1000000, the CountVectorizer will sort the vocabulary by 
# how often everything appears whether it be unigram/bigram/trigram 
# and it will cut it off after the defined count (1000000)
# most common ngrams
#veczr = CountVectorizer(ngram_range=(1,2), tokenizer=tokenize, max_features=800000)
#veczr = CountVectorizer(lowercase=False, stop_words='english', tokenizer=re_tokenizer, ngram_range=(1,2), min_df=2, max_df=0.95, max_features=1000000)
#veczr = CountVectorizer(lowercase=False, tokenizer=lemma_tokenizer, stop_words=stopwords, ngram_range=(1,2), min_df=2, max_df=0.8, max_features=2000000)
vec_tfidf = TfidfVectorizer(ngram_range=(1,5), tokenizer=lemma_tokenizer,  stop_words=stopwords, lowercase=True,
               min_df=4, max_df=0.8, strip_accents='unicode', sublinear_tf=True)
#trn_term_doc = veczr.fit_transform(train)
#val_term_doc = veczr.transform(val)
trn_term_doc = vec_tfidf.fit_transform(train)
val_term_doc = vec_tfidf.transform(val)

## Sentiment Analysis using Vader

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#reviews1 = pd.DataFrame(train, columns=['texts'])[0]
from time import time
t0 = time()
reviews = pd.DataFrame(train, columns=['texts'])
sentiments = pd.DataFrame(train_y, columns=['labels'])
sentiments = sentiments['labels'].replace(to_replace=[1, 0], value=['positive', 'negative'])

def predict_sentiment(review, printout, sentiment):
    analyzer = SentimentIntensityAnalyzer()
    #print(review)
    scores = analyzer.polarity_scores(review)  # scores has 4 values: neg, neu, pos, compound
    #print(scores)
    #pred = 'positive' if scores['compound'] > 0 else 'negative' if scores['compound'] < 0 else 'neutral'
    pred = 'positive' if scores['compound'] > 0 else 'negative'
    if (printout==True):
        print(review)
        print('neg',scores['neg'],', neu',scores['neu'],', pos',scores['pos'],', normalized_sum',scores['compound'])
        print('Predicted:', pred, ', Actual:', sentiment, '\n')
    return pred

#reviews = textdf.Text
#y = textdf.Sentiment
pred_list = [predict_sentiment(review, printout=False, sentiment=sentiments[0]) for review in reviews.texts]
predicted = pd.DataFrame(pred_list, columns=['pred'])

#print("Predicted:", predicted)
from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Evaluation results:\n' + 'Accuracy:', metrics.accuracy_score(sentiments, predicted))
cm = pd.DataFrame(confusion_matrix(sentiments, predicted, labels=['positive', 'negative']),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Evaluation results:
Accuracy: 0.6903
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8517                1483
actual negative                4711                5289
precision = 0.643861505896583 , recall = 0.8517 , F1-score = 0.7333390735319442
Elapsed Time: 159.507 seconds.


In [69]:
trn_term_doc.shape, val_term_doc.shape

((20000, 120819), (20000, 120819))

In [70]:
vocab = vec_tfidf.get_feature_names()
print(f'Vocabulary size: {len(vocab)}')

Vocabulary size: 120819


In [71]:
vocab[3100:3105]

['adverse', 'adversity', 'advert', 'advertise', 'advertised']

The **log-count ratio** $r$ for feature $f$ is the following:

f(a) = \frac{1}{2\pi i} \oint\frac{f(z)}{z-a}dz

$r = log \\frac{{ratio of feature $f$ in disaster tweets}{text{ratio of feature $f$ in no disaster tweets}}$


where features $f$ in the case of NLP are the tokens in our vocabulary (unigrams and bigrams). We will slowly build up to that equation in the next lines of code in order to understand better what is going on.

In [102]:
x = trn_term_doc.sign()
y = train_y
# x[y==1].sum(0) is to calculate the ratio of each feature in positive and no negative by dividing the toekn counts.

val_x = val_term_doc.sign()
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))
#r

pre_preds = val_x @r.T + b
#preds
preds = pre_preds.T>0
(preds == val_y).mean()

Regularized Logistic Regression where the features are trigrams

In [103]:
m = LogisticRegression(C=0.1, dual=False)
m.fit(x,y)
preds = m.predict(val_x)
(preds==val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

Results with TFIDF matrix:
Accuracy: 0.89065
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8920                1080
actual negative                1107                8893
precision = 0.8895980851700409 , recall = 0.892 , F1-score = 0.8907974234783043


Here is the log-count ratio r.

Now let us combine NB and Logistic regression together
is better than either
then use deep learning model to get a pretty much 
state-of-the-art result for structured learning 

r is a vector of rank 1 and length equals to number of features.
also logistic regression coefficients also rank 1 
and length equals to number of features.
calculating rank based on theory and based on data

In [104]:
# negative values are -ve feedback and positive values are +ve feedback
#r.shape, r


In [105]:
# e to the power of that, these are the ones we can compare to 1
# rather than to 0

#np.exp(r)

Here we fit regularized logistic regression where the fatures are the trigram's log-count ratios.

In [106]:
# take the term-document matrix and multiply by r
# every where 0 appears, 0 appears in x_nb and 
# every where 1 appears in TD, the r appears there
x_nb = x.multiply(r)
# we are going to use x_nb as independent variables instead in the
# logistic regression
m = LogisticRegression(dual=False, C=0.1)
m.fit(x_nb, y)

# do the samething for the validation set and get the predictions
# to get a better result
val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
#(preds.T==val_y).mean()
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

# regularization we start out with our 
# Loss = Cross entropy loss based on the predictions and the actuals + penalty
# L = X.C.(XW, y) + a W^2
# If you weights a large then that a.W^2 gets bigger and
# it drowns out X.C.(XW, Y) and we want it to be a good fit
# so we want to have as little regularization going on as we can get away with
# so we want to have lesser weights
# the weights that are close to zero were kind of not there.

# x_nb is creating kind of variation in weights that is going to cause that
# the W^2 to go up 
# When the input is already mulitplied by r is saying penalize things
# where we are varying from our NB prior
# I don't like to rely on the theory but if I have some theory then maybe
# we can use that as our starting point rather than starting off by assuming
# everything's equal, so our prior said we got this model quite Naive Bayes,
# and NB model said if NB assumptions were correct then r is the correct
# coefficient in this specific formulation because our prior is based
# on that theory

# that meeans, in a ML technique we can impute them with this kind of 
# Bayesian sense by starting out by incorporating the theoretical 
# expectations into the data that we feed to our model. 
# When we to do so, we don't need to regularize as much.
# If we regularize a lot, 

# C=1e-5 is the reciprocal of the amount of vectorization penalty so 
# we add lots of regularization by making C value very small
# that really hurts the accuracy, because it is really trying hard to get
# those weights down, the loss function is overwhelmed by the need to 
# reduce the weights and the need to make it predictive is 
# kind of now seen as totally unimportant
# So don't push the weights down so that you end up ignoring the terms
# but instead push them down so that you try to get rid of ignore 
# differences from our expectationi based on the NB formulation

# In deep learning we can get 94% accuracy,, but still particularly for a
# linear technique that's easy, fast and intuitive, this is pretty good and 
# we used trigrams instead of bigrams 

# Summary, it is mathematically equivalent to a normal logistic regression
# and the differene is actually in the regularization 


# Regularized Logistic Regresssion
# LR is prone to overfitting if you fit it with a very high order polymomial 
# features, like using a sigmoid function and it may end up with the hypothesis 
# whose decision boundary is a overly complex and extremely contorted function 
# that is not really a great hypothesis for a training set
# Also, in general, LR with a lot of features, it can end up with overfitting
# So we need to change the cost function of LR by adding the term lambda over 2m and sum from j = 1 to n for Theta j square
# This has the effect of penalizing the parameters Theta 1, 2 and so on...
# Even if we use high order poloymial with a lot of parameters, as long as we apply regularization and keep the parameters small
# the decision boundary will be more reasonable for separating out the positive and negative examples.
# Tuning parameter -- Lambda helps us explore how much we put emphasis on fitting the data versus making the magnitude of coefficient small
# When lambda is 0, we might get the standard likelihood solution i.e. unpenalized MLE solution
# When the lambda is infinity, the optimization becomes only care about penalizing the large coefficients of the parameters
# That means we are not fitting the data at all because of setting all the parameters to 0.
# We need a optimzied lambda which would balance the overfit and the magnitude of the parameter of the coefficients
# Finding the optimized lambda is called L2 regularized logistic regression
# Bias-Variance tradeoff is controlled by lambda
# When lambda is large can lead to large bias since theycan't fit the data very well and we have low variance
# When lambda is small, we get very good fit to the training data, so we have low bias, but we can have a very high variance
# if the data changed a little bit we get a completely different decision boundary 

# large can lead to large bias since theycan't fit the data very well and we have low variance





Results with TFIDF matrix:
Accuracy: 0.9034
Confusion matrix:
                  predicted positive  predicted negative
actual positive                9049                 951
actual negative                 981                9019
precision = 0.9021934197407777 , recall = 0.9049 , F1-score = 0.9035446829755367


In [107]:
m = LogisticRegression(C=0.1, dual=False)
m.fit(x,y)
preds = m.predict(val_x)
#print((preds==val_y).mean())
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

Results with TFIDF matrix:
Accuracy: 0.89065
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8920                1080
actual negative                1107                8893
precision = 0.8895980851700409 , recall = 0.892 , F1-score = 0.8907974234783043


In [52]:
from time import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1)
svc.fit(x, y)
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Elapsed Time: 943.586 seconds.


In [53]:
t0 = time()
preds = svc.predict(val_x)
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with TFIDF matrix:
Accuracy: 0.87835
Elapsed Time: 162.961 seconds.


In [54]:
#val_y_df = pval_y
#reviews = pd.DataFrame(train, columns=['texts'])
#val_y = sentiments['labels'].replace(to_replace=[1, 0], value=['positive', 'negative'])
#val_y_df

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Evaluation results:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))



Evaluation results:
Accuracy: 0.87835


In [37]:
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

Confusion matrix:
                  predicted positive  predicted negative
actual positive                8744                1256
actual negative                1177                8823
precision = 0.8813627658502167 , recall = 0.8744 , F1-score = 0.8778675769288691


In [38]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1e5)
svc.fit(x_nb, y)
print("Elapsed Time: %0.3f seconds." % (time() - t0))

t0 = time()
preds = svc.predict(val_x_nb)
print('Results with TFIDF matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
print("Elapsed Time: %0.3f seconds." % (time() - t0))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

Elapsed Time: 483.631 seconds.
Results with TFIDF matrix:
Accuracy: 0.8795
Elapsed Time: 119.355 seconds.
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8721                1279
actual negative                1131                8869
precision = 0.8852009744214373 , recall = 0.8721 , F1-score = 0.8786016522264759


In [39]:
from sklearn.metrics import accuracy_score
import math
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error

#pred_y = m.predict(x)
#(preds==val_y).mean()

mse=mean_squared_error(val_y, preds)
print("Mean Squared Error:",mse)
rmse=math.sqrt(mse)
print("Root Mean Squared Error:", rmse)

mse=mean_absolute_error(val_y, preds)
print("Mean Absolute Error:",1-mse)
#preds

Mean Squared Error: 0.1205
Root Mean Squared Error: 0.34713109915419565
Mean Absolute Error: 0.8795
